In [14]:
# example of training a final classification model
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets.samples_generator import make_blobs

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import pickle

import numpy as np
import pandas as pd

In [2]:
path = '../Entrenamientos/Procesado_Tramos.xlsx'
df = pd.read_excel(path)
df = df[df["porc"]!=np.inf]
df["puntos_en_tramo"]= df["tramo_final"]-df["tramo_inicial"]
df = df[df["puntos_en_tramo"]>9]

df2 = df[["pwr","porc","viento_aparente","spd"]]

dataset = df2.values

In [3]:
# generate 2d classification dataset
X = dataset[:,0:3]
Y = dataset[:,3]

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [5]:
#standarScaler

#scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
#scalarX, scalarY = StandardScaler(), StandardScaler()
scalarX, scalarY = RobustScaler(), RobustScaler()

scalarX.fit(X)
#scalarY.fit(Y.reshape(len(Y),1))
X = scalarX.transform(X)

In [6]:
#Probar por diferentes funciones de activacion y con regularizacion

# define and fit the final model
model = Sequential()
model.add(Dense(4, input_dim=3, activation='relu'))

model.add(Dense(60, activation='relu'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, Y, epochs=1000, verbose=0)

In [7]:
Xnew = scalarX.transform(X_test)


# make a prediction
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
        print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))


X=[-0.30660975 -1.00795275  0.01751247], Predicted=[46.596592]
X=[0.09026737 0.39460717 0.12175835], Predicted=[26.343288]
X=[ 0.85388585  0.65633681 -0.02679492], Predicted=[25.379795]
X=[ 0.56968894  0.32276831 -0.50364034], Predicted=[28.239838]
X=[0.4338394  0.35518791 0.55186448], Predicted=[30.526205]
X=[-0.07442161 -0.08541085  1.53711416], Predicted=[33.70247]
X=[-0.22714284 -0.1213728   0.43658106], Predicted=[34.41341]
X=[ 0.61379527  0.55302236 -1.12726495], Predicted=[28.393986]
X=[-0.47421712 -1.00428001  0.481916  ], Predicted=[46.81461]
X=[-0.02334338 -0.13279544  0.43534838], Predicted=[35.649094]
X=[ 0.41908224 -0.31466816  0.38679217], Predicted=[40.707645]
X=[-1.32059448 -1.76724194 -0.30372868], Predicted=[48.018646]
X=[-0.71269817 -0.58478372 -0.53044285], Predicted=[35.272682]
X=[0.26215717 0.35668735 0.44173137], Predicted=[29.162987]
X=[ 0.66934475  0.95696698 -0.02209967], Predicted=[20.429722]
X=[-0.08920702 -0.15722822 -0.51039003], Predicted=[31.776154]
X=[ 

In [8]:
test_val = pd.DataFrame(X_test)
test_val.columns = (["pwr","porc","viento_aparente"])
test_val["spd"] = Y_test
test_val["y_pred"] = ynew
test_val["diff"] = test_val["spd"]-test_val["y_pred"]
test_val.head()

,pwr,porc,viento_aparente,spd,y_pred,diff
0,142.463180,-3.837103,-0.867769,42.144151,46.596592,-4.452441
1,184.873741,1.079441,0.761328,26.432877,26.343288,0.089588
2,266.474534,1.996910,-1.560181,28.776319,25.379795,3.396524
3,236.105059,0.827617,-9.012058,31.143334,28.239838,2.903496
4,221.588084,0.941261,7.482789,26.676923,30.526205,-3.849282


In [9]:
print("Predicciones:")
print(test_val["y_pred"].mean())
print(test_val["y_pred"].max())
print(test_val["y_pred"].min())

print("Diferencias:")
print(test_val["diff"].mean())
print(test_val["diff"].max())
print(test_val["diff"].min())

Predicciones:
29.255605697631836
48.018646
11.706385
Diferencias:
0.025786558614786268
9.61514143493816
-16.665178560875145


In [10]:
test_val[(test_val["diff"]>8) | (test_val["diff"]<-10)]

,pwr,porc,viento_aparente,spd,y_pred,diff
10,220.011123,-1.406858,4.903130,24.042467,40.707645,-16.665179
11,34.108079,-6.498721,-5.887949,35.093972,48.018646,-12.924674
18,157.716371,0.371855,-10.881710,35.510085,25.894943,9.615141
58,95.396327,2.311267,-1.141445,1.718992,14.821535,-13.102544


In [11]:
len(test_val)

65

In [23]:
pickle.dump(scalarX, open("Scaler.model", 'wb'))


model.save("TF_Tramos_2.model")
#Para cargar el modelo
#https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model